# Recruit Restaurant Visitor Forecasting
written by T.Bayz @5 Oct 2019

In [7]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [26]:
# load data
data = {
    "ar": pd.read_csv("data/air_reserve.csv",engine="python"),
    "hr": pd.read_csv("data/hpg_reserve.csv",engine="python"),
    "as": pd.read_csv("data/air_store_info.csv",engine="python"),
    "hs": pd.read_csv("data/hpg_store_info.csv",engine="python"),
    "id": pd.read_csv("data/store_id_relation.csv",engine="python"),
    "date_info": pd.read_csv("data/date_info.csv",engine="python").rename(columns={'calendar_date':'visit_date'}),
    "av": pd.read_csv("data/air_visit_data.csv",engine="python"),
    "test": pd.read_csv("data/sample_submission.csv",engine="python")
}

##### 店舗情報処理(air,hpgそれぞれ)

In [33]:
import copy

train = {}

# データを持ってくる
train["hpg"] = copy.copy(data['hr'])
train["air"] = copy.copy(data['ar'])

# 店舗IDを揃える hpg => airに変換する
train["hpg"] = pd.merge(train["hpg"], data['id'], how='inner', on=['hpg_store_id'])
train["hpg"] = train["hpg"].drop("hpg_store_id",axis=1)

# 店舗IDのカラム名を揃える
for df in ["hpg","air"]:
    train[df] = train[df].rename(columns={"air_store_id":"id"})

for df in ["hpg","air"]:
    train[df] = pd.merge(train[df], data["as"], how='inner', left_on="id", right_on="air_store_id")
    train[df]["id"] = train[df]["id"].str.split("_",expand=True)[1]
    train[df] = train[df].drop(["air_area_name","air_store_id"],axis=1)

##### 時間情報処理(air,hpgそれぞれ)

In [34]:
# タイムスタンプ形式にする
for df in ["hpg","air"]:
    train[df]["visit_datetime"] = pd.to_datetime(train[df]["visit_datetime"])
    train[df]["reserve_datetime"] = pd.to_datetime(train[df]["reserve_datetime"])

for df in ["hpg","air"]:
    train[df]["dow"] = train[df]["visit_datetime"].dt.dayofweek
    train[df]["year"] = train[df]["visit_datetime"].dt.year
    train[df]["month"] = train[df]["visit_datetime"].dt.month
    train[df]["day"] = train[df]["visit_datetime"].dt.day

In [35]:
for df in ["hpg","air"]:
    train[df].to_csv(df+".csv")